In [2]:
using DataFrames,PythonPlot,StatsBase,Random,Distributions,CSV,DSP,LinearAlgebra
PythonPlot.svg(true)
include("model.jl")

FIG_PATH = "/Users/elevien/Dartmouth College Dropbox/Ethan Levien/RESEARCH/ACTIVE/cell-size/linear_model/figures"

"/Users/elevien/Dartmouth College Dropbox/Ethan Levien/RESEARCH/ACTIVE/cell-size/linear_model/figures"

In [ ]:
function pmap(r,t,a,bz,bθ,uz)
    A = [0.0 uz;-a -bz]
    expm(A*t)*r + exp
end

In [ ]:
samples = 10000
A = [-0.3 -0.0;0.0 -1.0]
print(eigvals(A))
u = randn(2,samples)
b = randn(2,samples)
evals = [max(real.(eigvals([0.0 u[:,i]'; b[:,i] A]))...) for i in 1:samples]
prods = [-(u[:,i])'*inv(A)*b[:,i] for i in 1:samples];

fig,ax = subplots(figsize=(3,3))

ax.plot(prods,evals,".")
xrng = minimum(prods):0.01:maximum(prods)
ax.plot(xrng,-real.(1 .- sqrt.(Complex.(1 .+ 4 .* xrng)))/2,"k--")
ax.fill_between(xrng,-ones(length(xrng)),zeros(length(xrng)),color="gray",alpha=0.3,lw=0)
ax.fill_betweenx(xrng,-ones(length(xrng)),zeros(length(xrng)),color="C1",alpha=0.05,lw=0)
ax.set_xlim([-0.5,0.5])
ax.set_ylim([-0.6,0.5])
fig